In [ ]:
!pip install git+https://github.com/entelecheia/eKoNLPy.git
!pip install eKoNLPy

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from nltk.util import everygrams
from ekonlpy.sentiment import MPCK
from ekonlpy.sentiment import MPKO
import re
from tqdm.notebook import tqdm

In [ ]:
# 파일 불러오기
df_conference = pd.read_csv('/content/drive/MyDrive/BOK_금리예측_비공개/data/conference.csv')
df_conference

,Unnamed: 0,DATE,CONTENT
0,0,2022-10-12,공 보 관-지금부터 2022년 10월 통화정책방향에 대한 이창용 한국은행 총재의 기...
1,1,2022-08-25,부공보관 - 지금부터 2022년 8월 통화정책방향에 대한 이창용 한국은행 총재의...
2,2,2022-07-13,공 보 관 - 지금부터 2022년 7월 통화정책방향에 대한 이창용 한국은행 총재...
3,3,2022-06-21,공 보 관 - 지금부터 2022년 6월 물가안정목표 운영상황 점검 설명회를 시작...
4,4,2022-05-26,공 보 관 - 지금부터 2022년 5월 통화정책방향에 대한 이창용 한국은행 총재...
5,5,2022-04-14,공 보 관 - 지금부터 2022년 4월 통화정책방향에 대한 기자간담회를 시작하겠...
6,6,2022-02-24,공 보 관 - 지금부터 2022년 2월 통화정책방향에 대한 이주열 한국은행 총재...
7,7,2022-01-14,부공보관 - 지금부터 2022년 1월 통화정책방향에 대한 이주열 한국은행 총재님...
8,8,2021-12-16,공 보 관 - 지금부터 물가안정목표 운영상황에 대한 이주열 한국은행 총재님의 기...
9,9,2021-11-25,공 보 관 - 지금부터 2021년 11월 통화정책방향에 대한 이주열 한국은행 총...


In [ ]:
mpck = MPCK()
df_conference['ngram_prep'] = ''
for idx in tqdm(df_conference.index):
        content = df_conference.loc[idx, 'CONTENT']
        content = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣,. ]', '', content)
        content = re.sub('\d+.\d*', '',content)
        content = re.sub("([a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", '', content)
        content = re.sub("\.+\.", '.', content)
        content = re.sub(' +', ' ', content)
        cnt_temp = []
        cnt_list = content.split('.')
        for cnt in cnt_list:
            if (cnt not in ['', ' ']) and (len(cnt) >  10) :
                cnt_temp.append(cnt)
        # print(cnt_temp[:5])
        ngram_list = []
        
        for ng in cnt_temp:
            tokens = mpck.tokenize(ng)
            ngrams = mpck.ngramize(tokens)
            if ngrams != []:
                ngram_list.append(ngrams)
        if ngram_list != [] :
            df_conference.loc[idx, 'ngram_prep'] = str(ngram_list)
df_conference2 = df_conference.drop(['CONTENT'], axis=1)
df_conference2.to_csv(f'conference_ngramize.csv', index=False)

  0%|          | 0/55 [00:00<?, ?it/s]